# Summary
- The DOC2VEC is trained on window=21, word_count =1, and iter =50 to generate 16x16 size vectors
- This notebook is not very time consuming. It runs about 1 hour in my desktop (2 XEON and 40G DDR)
- During processing, this notebook consumes about 10G memory.

In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:

# Load stop-words
stop_words = set(stopwords.words('english'))
stop_words.update(('name','surname','given','first'))

# Initialize tokenizer
regexp_tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
# Tokenize a document to words
def tokenize(document, rebuild_document=True):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [wordnet_lemmatizer.lemmatize(t.lower()) 
                  for t in regexp_tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    if rebuild_document:
        return ' '.join(words).strip()
    else:
        return words

In [4]:
# load tokenized file
p_file = 'taggedDoc'
with open(p_file, 'rb') as fin:
    taggedDoc = pickle.load(fin)

In [5]:
start = time.time()

# Train Doc2Vec model
# https://arxiv.org/pdf/1405.4053v2.pdf
doc2vecSize = 256

doc2vecModel = Doc2Vec(documents=taggedDoc, size=doc2vecSize, window=21, 
                        min_count=1, iter=50, workers=multiprocessing.cpu_count())
doc2vecModel.save('doc2vecModel')
doc2vecModel.init_sims(replace=False)

end = time.time()
elapsed = end - start
print ("Time consumed to train the Doc2Vec: ", elapsed)

Time consumed to train the Doc2Vec:  4157.049939393997


In [6]:
p_file = 'data_X'

with open(p_file, 'rb') as fin:
    data_X = pickle.load(fin)

In [7]:
# Vectorize documents
vectors = []

for i in tqdm.tqdm(range(len(data_X))):
    vectors.append(doc2vecModel.infer_vector(data_X[i].split(' ')))

100%|██████████| 19111/19111 [26:57<00:00, 11.82it/s]


In [8]:
p_file = 'RCT_Vectors'

with open(p_file, 'wb') as fout:
    pickle.dump(vectors, fout)